In [1]:
from scipy import stats
import time
# import requests
from bs4 import BeautifulSoup
import threading
import pandas as pd
import tqdm
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler, PowerTransformer
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle
import sys

   
sys.path.append("/home/td/Documents/sports_predictor")
sys.path.append("/home/td/Documents/sports_predictor/nba")

from process_data import DataManager

base_url = 'https://www.basketball-reference.com/'
day_scores_base_url = 'https://www.basketball-reference.com/boxscores/?month={month}&day={day}&year={year}'
data_path = r'/media/td/Samsung_T5/sports/nba'
# data_path = r'C:\Users\TristanDelforge\Documents\sports_predictor\nba'
db_name = 'nba_db'
box_score_link_table_name = 'boxscore_links'

box_score_details_table_name = 'boxscore_details'
processed_team_data_table_name = 'processed_team_data'
player_detail_table_name = 'player_details'
processed_player_data_table_name = 'processed_player_data'
aggregated_player_data_table_name = 'aggregated_player_data'
combined_feature_file_data_table_name = 'combined_feature_file'
past_n_game_dataset_table_name = 'past_n_game_dataset'
target = 'win'

date_record_pickle_file_name = 'scraped_dates'
box_score_record_pickle_file_name = 'scraped_games'
max_tries = 5
file_lock = threading.Lock()

starting_rating = 1000
rating_k_factor = 100
rating_floor = 100
rating_ceiling = 10000
rating_d = 1000
k_min_sensitivity = 1

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [2]:
# dm = DataManager(testing = 2000)
# dm.update_raw_datasets()
# dm.build_timeseries(4, False)
# dm.combine_timeseries(4, False)
# x1, x2, y, cols = dm.get_labeled_data(4, False)
# print(x1.shape, x2.shape, y.shape, len(cols))

In [3]:
# np.isnan(x1).astype(int).sum()

In [4]:
# x2_df = pd.DataFrame(data=x2,
#                     columns = cols)
# x2_df.isna().sum().sum()

In [5]:
# x2_df.head(10)

In [72]:
team_data = pd.read_csv('{data_path}/{db_name}.csv'.format(data_path=data_path,db_name=box_score_details_table_name),sep='|', low_memory=False)
player_data = pd.read_csv('{data_path}/{db_name}.csv'.format(data_path=data_path,db_name=player_detail_table_name), sep='|', low_memory=False)

team_data.shape, player_data.shape

((55508, 45), (680665, 48))

In [78]:
player_data['date_str'] = player_data.apply(
            lambda x: str(x['year']).zfill(4) + '-' + str(x['month']).zfill(2) + '-' + str(x['day']).zfill(2), axis=1)
player_data['game_key'] = player_data.apply(
            lambda x: str(sorted([str(x['date_str']), str(x['team_tag']), str(x['opponent_tag'])])), axis=1)
player_data['team_game_key'] = player_data.apply(
            lambda x: str([str(x['date_str']), str(x['team_tag']), str(x['opponent_tag'])]), axis=1)
player_data.head()

,ast,ast_pct,blk,blk_pct,day,def_rtg,drb,drb_pct,efg_pct,fg,fg3,fg3_pct,fg3a,fg3a_per_fga_pct,fg_pct,fga,ft,ft_pct,fta,fta_per_fga_pct,location,month,mp,off_rtg,opponent_link,opponent_name,opponent_tag,orb,orb_pct,pf,player_link,player_name,plus_minus,pts,reason,stl,stl_pct,team_link,team_name,team_tag,tov,tov_pct,trb,trb_pct,ts_pct,usg_pct,win,year,date_str,game_key,team_game_key
0,6.0,46.0,0.0,0.0,8,108.0,2.0,6.8,0.417,2.0,1.0,1.000,1.0,0.167,0.333,6.0,6.0,1.000,6.0,1.000,NaN,11,37:38,121.0,https://www.basketball-reference.com//teams/SA...,San Antonio Spurs,sas,0.0,0.0,3.0,https://www.basketball-reference.com//players/...,Howard Eisley,-5.0,11.0,NaN,0.0,0.0,https://www.basketball-reference.com//teams/UT...,Utah Jazz,uta,2.0,18.8,2.0,3.2,0.637,13.7,0,1997,1997-11-08,"['1997-11-08', 'sas', 'uta']","['1997-11-08', 'uta', 'sas']"
1,3.0,23.7,0.0,0.0,8,102.0,4.0,14.0,0.250,2.0,1.0,0.333,3.0,0.300,0.200,10.0,7.0,0.875,8.0,0.800,NaN,11,37:36,85.0,https://www.basketball-reference.com//teams/SA...,San Antonio Spurs,sas,3.0,9.5,5.0,https://www.basketball-reference.com//players/...,Jeff Hornacek,0.0,12.0,NaN,1.0,1.5,https://www.basketball-reference.com//teams/UT...,Utah Jazz,uta,4.0,22.8,7.0,11.6,0.444,23.1,0,1997,1997-11-08,"['1997-11-08', 'sas', 'uta']","['1997-11-08', 'uta', 'sas']"
2,1.0,13.8,1.0,2.0,8,101.0,6.0,21.6,0.467,7.0,0.0,NaN,0.0,0.000,0.467,15.0,12.0,0.800,15.0,1.000,NaN,11,36:17,94.0,https://www.basketball-reference.com//teams/SA...,San Antonio Spurs,sas,1.0,3.3,5.0,https://www.basketball-reference.com//players/...,Karl Malone,-5.0,26.0,NaN,0.0,0.0,https://www.basketball-reference.com//teams/UT...,Utah Jazz,uta,7.0,24.5,7.0,12.0,0.602,38.8,0,1997,1997-11-08,"['1997-11-08', 'sas', 'uta']","['1997-11-08', 'uta', 'sas']"
3,1.0,8.9,0.0,0.0,8,96.0,2.0,8.4,0.300,1.0,1.0,0.333,3.0,0.600,0.200,5.0,2.0,1.000,2.0,0.400,NaN,11,30:42,93.0,https://www.basketball-reference.com//teams/SA...,San Antonio Spurs,sas,2.0,7.6,4.0,https://www.basketball-reference.com//players/...,Bryon Russell,0.0,5.0,NaN,3.0,5.4,https://www.basketball-reference.com//teams/UT...,Utah Jazz,uta,1.0,14.5,4.0,7.9,0.425,10.8,0,1997,1997-11-08,"['1997-11-08', 'sas', 'uta']","['1997-11-08', 'uta', 'sas']"
4,0.0,0.0,1.0,3.8,8,89.0,3.0,20.5,0.000,0.0,0.0,NaN,0.0,0.000,0.000,3.0,1.0,0.250,4.0,1.333,NaN,11,18:48,37.0,https://www.basketball-reference.com//teams/SA...,San Antonio Spurs,sas,1.0,6.2,5.0,https://www.basketball-reference.com//players/...,Greg Ostertag,-6.0,1.0,NaN,2.0,5.8,https://www.basketball-reference.com//teams/UT...,Utah Jazz,uta,0.0,0.0,4.0,13.0,0.105,12.2,0,1997,1997-11-08,"['1997-11-08', 'sas', 'uta']","['1997-11-08', 'uta', 'sas']"


In [79]:
player_data['team_game_key'].value_counts()

['2003-04-19', 'mil', 'njn']    18
['2003-01-17', 'tor', 'njn']    17
['2001-02-22', 'was', 'sac']    17
['2009-02-23', 'sac', 'noh']    17
['2010-02-16', 'dal', 'okc']    16
                                ..
['2009-02-20', 'sac', 'mem']     9
['2003-04-16', 'tor', 'cle']     8
['2003-02-21', 'sea', 'mil']     8
['1999-03-12', 'njn', 'phi']     8
['2009-02-20', 'chi', 'den']     8
Name: team_game_key, Length: 55508, dtype: int64

In [81]:
player_data['team_game_key'].value_counts().describe(percentiles=[.9, .95, .99])

count    55508.000000
mean        12.262467
std          0.552358
min          8.000000
50%         12.000000
90%         13.000000
95%         13.000000
99%         13.000000
max         18.000000
Name: team_game_key, dtype: float64

In [86]:

def parse_float(s):
    try:
        return float(s)
    except:
        pass

def process_minutes_played(s):
    s_split = str(s).split(':')
    if len(s_split) == 2:
        minutes_parsed = parse_float(s_split[0])
        seconds_parsed = parse_float(s_split[1])
        if minutes_parsed and seconds_parsed:
            total_minutes_played = minutes_parsed + (seconds_parsed/60)
            return total_minutes_played

    return 0
    

In [87]:
player_data['mp'] = player_data['mp'].apply(process_minutes_played)

In [88]:
player_data = player_data.sort_values('date_str')

In [92]:
players = set(player_data['player_link'])

period_size = 32
for t in tqdm.tqdm(players):
    player_data.loc[player_data['player_link'] == t, 'mp_trailing_avg_{}'.format(period_size)] = player_data[player_data['player_link'] == t].shift(periods=1).rolling(window=period_size)['mp'].mean()
    

100%|██████████| 2076/2076 [02:18<00:00, 15.00it/s]


In [102]:
team_game_ids = sorted(list(set(player_data['team_game_key'])))
team_game_ids_vc = player_data['team_game_key'].value_counts()

columns = ['ast', 'ast_pct', 'blk', 'blk_pct', 'def_rtg', 'drb', 'drb_pct', 'efg_pct',
                                            'fg', 'fg3', 'fg3_pct', 'fg3a', 'fg3a_per_fga_pct', 'fg_pct', 'fga', 'ft',
                                            'ft_pct',
                                            'fta', 'fta_per_fga_pct', 'mp', 'off_rtg', 'orb', 'orb_pct', 'pf',
                                            'plus_minus', 'pts', 'stl', 'stl_pct', 'tov', 'tov_pct', 'trb', 'trb_pct',
                                            'ts_pct',
                                            'usg_pct']

num_of_players = 12

counter = 0
result_dict = dict()
for i in tqdm.tqdm(team_game_ids):
    sub_df = player_data.iloc[counter:counter+team_game_ids_vc[i]]
    sub_df = sub_df.copy()
    assert sub_df.shape[0] == team_game_ids_vc[i]
    
    sub_df = sub_df.sort_values('mp_trailing_avg_32', ascending = False)
    sub_df = sub_df.reset_index()
    
    
    res_arrays = []
    for j in range(num_of_players):
        if j < sub_df.shape[0]:
            s = sub_df.loc[j]
            s = s[columns]
            s = s.values
            res_arrays.append(s)
        else:
            res_arrays.append(np.array([0 for _ in columns]))
        
    result_dict[i] = res_arrays
    break
    
    



  0%|          | 0/55508 [00:00<?, ?it/s]


In [106]:
np.hstack(result_dict["['1997-11-08', 'atl', 'cle']"]).shape

(408,)

In [8]:
# player_data.head()

In [9]:
# team_data_corr = team_data.corr()
# team_data_corr = team_data_corr[(team_data_corr['win'] > .2)|(team_data_corr['win'] < -.08)]
# team_data_corr.sort_values('win')[['win']]

In [10]:
# player_data_corr = player_data.corr()
# player_data_corr = player_data_corr[(player_data_corr['win'] > .05)|(player_data_corr['win'] < -.05)]
# player_data_corr.sort_values('win')[['win']]

In [11]:

# past_n_game_obj = None
# with open('{data_path}/{db_name}.pkl'.format(data_path=data_path,
#                                                              db_name='past_n_game_dataset_32_False'), 'rb') as f:
#     past_n_game_dataset = pickle.load(f)
# past_n_game_dataset.keys()


In [12]:
# team_data[(team_data['team_tag'] == 'pho')&(team_data['month'] == 12)&(team_data['year'] == 2017)][['month', 'day', 'ast']].head(20)

In [13]:
# initial_team_data_columns = ['ast', 'ast_pct', 'blk', 'blk_pct', 'def_rtg', 'drb', 'drb_pct', 'efg_pct',
#                                           'fg', 'fg3', 'fg3_pct', 'fg3a', 'fg3a_per_fga_pct', 'fg_pct', 'fga', 'ft',
#                                           'ft_pct',
#                                           'fta', 'fta_per_fga_pct', 'mp', 'off_rtg', 'orb', 'orb_pct', 'pf',
#                                           'plus_minus', 'pts', 'stl', 'stl_pct', 'tov', 'tov_pct', 'trb', 'trb_pct',
#                                           'ts_pct',
#                                           'usg_pct', 'home', 'r1', 'r2', 'r3', 'r4']

# pd.DataFrame(data=past_n_game_dataset['pho']["['2017-12-31', 'phi', 'pho']"])


In [110]:
results = pd.read_csv(f'{data_path}/nn_architectures.csv')
results = results.replace(True, 1).replace(False, 0)

results['block_1_dropout_amount'] = results['block_1_dropout_amount'].fillna(.2)
results['block_2_dropout_amount'] = results['block_2_dropout_amount'].fillna(.2)
results['merge_block_dropout_amount'] = results['merge_block_dropout_amount'].fillna(.2)
results['top_block_dropout_amount'] = results['top_block_dropout_amount'].fillna(.2)

results.sort_values('accuracy', ascending = False).head(20)

,convolutional_kernel_size,convolutional_pool_size,pooling_algorithm,convolution_type,recurrent_type,use_x1,use_x2,optimizer_algorithm,conv_layers_per_pooling_layer,batchnorm,block_1_activations,block_2_activations,merge_activations,top_block_activations,block_1_dropout,block_2_dropout,merge_block_dropout,top_block_dropout,merge_block_width,block_2_width,top_block_width,recurrent_layers_width,block_1_dense_layers_width,merge_block_layers,block_2_layers,convolutional_filters,dense_top_layers,convolutional_layers,recurrent_layers,block_1_dense_layers,block_1_dropout_amount,block_2_dropout_amount,merge_block_dropout_amount,top_block_dropout_amount,input_shape1,input_shape2,history_lengths,team_encoding_type,team_encoding_size,player_encoding_type,player_encoding_size,rotate_history_data,num_of_params,model_training_time,accuracy
1,1,2,layers.MaxPooling1D,layers.SeparableConv1D,layers.GRU,1,1,optimizers.Adamax(),1,0,softsign,selu,relu,softsign,layers.GaussianDropout,layers.Dropout,layers.Dropout,NaN,38,96,246,22,183,1,2,117,0,1,3,2,0.653252,0.602781,0.288526,0.096486,"(32, 12)","(592,)",32.0,pca,4.0,NaN,NaN,0.0,219927.0,123.322812,0.658997
5,3,4,NaN,layers.SeparableConv1D,layers.SimpleRNN,1,1,optimizers.Adamax(),3,1,sigmoid,softsign,relu,sigmoid,layers.Dropout,NaN,layers.AlphaDropout,layers.GaussianDropout,77,118,163,61,220,0,0,78,3,2,0,0,0.347719,0.159080,0.532988,0.170623,"(48, 32)","(592,)",32.0,pca,16.0,NaN,NaN,1.0,676605.0,65.000197,0.655493
10,4,1,layers.MaxPooling1D,layers.Conv1D,layers.GRU,1,1,optimizers.Adagrad(),2,1,hard_sigmoid,elu,tanh,hard_sigmoid,layers.GaussianDropout,layers.AlphaDropout,layers.Dropout,layers.GaussianDropout,249,52,238,83,106,1,1,74,2,1,0,3,0.156631,0.139421,0.363943,0.356394,"(32, 6)","(592,)",32.0,pca,2.0,NaN,NaN,0.0,769785.0,40.588551,0.649093
19,5,3,NaN,layers.LocallyConnected1D,layers.GRU,1,1,optimizers.SGD(),2,0,softsign,relu,hard_sigmoid,hard_sigmoid,layers.GaussianDropout,layers.AlphaDropout,layers.Dropout,layers.Dropout,132,241,208,28,57,1,0,127,2,3,1,1,0.002320,0.247882,0.148324,0.377444,"(32, 96)","(592,)",32.0,pca,32.0,NaN,NaN,0.0,6175868.0,1309.098007,0.639799
15,9,4,layers.MaxPooling1D,layers.Conv1D,layers.LSTM,1,0,optimizers.Adamax(),0,1,exponential,exponential,sigmoid,elu,layers.AlphaDropout,layers.GaussianDropout,layers.GaussianNoise,layers.GaussianDropout,56,78,140,141,163,1,1,123,2,0,2,0,0.674013,0.707044,0.241520,0.335877,"(32, 48)","(592,)",32.0,pca,16.0,NaN,NaN,0.0,549414.0,78.162627,0.639647
20,7,1,NaN,layers.SeparableConv1D,layers.SimpleRNN,1,0,optimizers.Adadelta(),2,0,hard_sigmoid,sigmoid,relu,softplus,NaN,layers.Dropout,layers.GaussianDropout,layers.GaussianNoise,155,177,159,63,138,1,1,68,2,0,1,3,0.427702,0.798331,0.779186,0.438882,"(32, 12)","(592,)",32.0,pca,4.0,NaN,NaN,0.0,480871.0,12.644933,0.638275
3,9,4,NaN,layers.SeparableConv1D,layers.LSTM,1,0,optimizers.RMSprop(),2,0,tanh,softplus,softsign,exponential,layers.GaussianDropout,layers.Dropout,NaN,layers.GaussianDropout,154,161,172,202,201,0,0,81,0,0,0,1,0.360424,0.158856,0.621077,0.374414,"(48, 32)","(592,)",32.0,pca,16.0,NaN,NaN,1.0,309341.0,3.377141,0.634618
0,6,4,layers.MaxPooling1D,layers.LocallyConnected1D,layers.GRU,1,0,optimizers.RMSprop(),1,0,softsign,hard_sigmoid,hard_sigmoid,softplus,layers.Dropout,NaN,NaN,layers.AlphaDropout,172,189,61,119,76,0,0,120,3,1,0,0,0.615499,0.728111,0.229223,0.336617,"(32, 96)","(592,)",32.0,pca,32.0,NaN,NaN,0.0,54597.0,6.953111,0.625324
9,9,3,NaN,layers.LocallyConnected1D,layers.LSTM,1,0,optimizers.Adamax(),2,1,exponential,softsign,softplus,selu,NaN,layers.AlphaDropout,layers.GaussianNoise,NaN,134,164,55,148,185,0,2,42,0,0,1,1,0.687691,0.540073,0.374055,0.575997,"(12, 32)","(592,)",32.0,pca,4.0,NaN,NaN,1.0,183633.0,20.280239,0.624105
18,1,1,NaN,layers.Conv1D,layers.SimpleRNN,1,0,optimizers.Nadam(),1,0,elu,hard_sigmoid,softplus,relu,layers.GaussianNoise,layers.GaussianDropout,layers.GaussianNoise,NaN,240,103,112,49,75,0,0,85,2,3,3,2,0.121574,0.714690,0.583262,0.275614,"(32, 96)","

In [108]:
results.shape

(19, 45)

In [112]:
df = pd.DataFrame(data = [[1, 3], [2, 2], [3, 1]],
                 columns = ['a', 'b'])
df.head()

,a,b
0,1,3
1,2,2
2,3,1


In [113]:
df.sort_values(['a', 'b'])

,a,b
0,1,3
1,2,2
2,3,1


In [ ]:
results = pd.read_csv(f'{data_path}/nn_architectures.csv')
results.sort_values('accuracy', ascending = False).head(10)

In [ ]:
results = pd.read_csv(f'{data_path}/nn_architectures.csv')
results[results['history_lengths'] == 4].sort_values('accuracy', ascending = False).head(20)

In [ ]:
results = pd.read_csv(f'{data_path}/nn_architectures.csv')
results[results['history_lengths'] == 8].sort_values('accuracy', ascending = False).head(20)

In [ ]:
results = pd.read_csv(f'{data_path}/nn_architectures.csv')
results[results['history_lengths'] == 16].sort_values('accuracy', ascending = False).head(20)

In [ ]:
results = pd.read_csv(f'{data_path}/nn_architectures.csv')
results[results['history_lengths'] == 32].sort_values('accuracy', ascending = False).head(20)